In [1]:
import ee
import geemap

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize(project="coastal-well-476404-m6")

In [5]:
print(ee.__version__)
print(geemap.__version__)

1.6.13
0.36.6


### Import Ward File of Delhi

In [6]:
import geopandas as gpd

PATH_WARDS = "Delhi/Delhi_wards.shp"
gdf = gpd.read_file(PATH_WARDS)
gdf = gdf.to_crs(4326) 

### Changed to gee format

In [7]:
geojson_str = geemap.gdf_to_geojson(gdf)     # first change to geojson
wards_fc = geemap.geojson_to_ee(geojson_str) # EE FeatureCollection

### Plotting of Delhi Ward File

In [8]:
wards_geom = wards_fc.geometry()

# Visualize boundary
Map = geemap.Map(center=[28.6, 77.2], zoom=9)
Map.addLayer(wards_fc, {'color': 'blue'}, 'Delhi Wards Boundary')
Map

Map(center=[28.6, 77.2], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', …

### Plotting of VIIRS

In [9]:
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
    .filterDate('2015-01-01', '2018-12-31') \
    .filterBounds(wards_geom)

nighttime = dataset.select('avg_rad').mean().clip(wards_geom)

# ---- Visualization Parameters ----
nighttime_vis = {
    'min': 0.0,
    'max': 60.0,
    "palette": ['white', 'yellow', 'cyan', 'purple', 'black']
}

# ---- Create Map and Add Layers ----
Map = geemap.Map(center=[28.6, 77.2], zoom=9)
Map.addLayer(wards_fc, {'color': 'blue'}, 'Delhi Wards Boundary')
Map.addLayer(nighttime, nighttime_vis, 'VIIRS Nighttime Lights (2015-2018)')
Map.add_colorbar(nighttime_vis, label='Mean Radiance (nW/cm²/sr)')
Map

Map(center=[28.6, 77.2], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', …

### Plotting of surface reflectance by Landsat 9

In [10]:
# ---- Step 1: Cloud and shadow masking function ----
def mask_l2_sr(image):
    """Masks clouds and shadows using QA_PIXEL band."""
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).neq(0)    # Bit 3: cloud
    shadow = qa.bitwiseAnd(1 << 4).neq(0)   # Bit 4: shadow
    mask = cloud.Or(shadow).Not()
    return image.updateMask(mask)

# ---- Step 2: Apply scaling factors ----
def apply_scale_factors(image):
    """Applies scaling to optical and thermal bands."""
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands.rename('ST_B10'), None, True)

# ---- Step 3: Load Landsat 8 Collection ----
col = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(wards_geom)
    .filterDate("2015-01-01", "2022-12-31")
    .map(mask_l2_sr)
    .map(apply_scale_factors)
)

# ---- Step 4: Create a median composite ----
img = col.median().clip(wards_geom)

# ---- Step 5: Compute NDVI ----
# NDVI = (NIR - RED) / (NIR + RED)
ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# ---- Step 6: Compute mean NDVI for each ward ----
def add_mean_ndvi(f):
    stats = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=f.geometry(),
        scale=30,
        maxPixels=1e13,
        tileScale=4
    )
    return f.set('NDVI_mean', stats.get('NDVI'))

# Add ward-wise NDVI mean
zr = wards_fc.map(add_mean_ndvi)

# ---- Step 7: Visualization ----
ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    "palette": ["brown", "yellow", "lightgreen", "green", "darkgreen"]
}

m = geemap.Map(center=[28.61, 77.23], zoom=9)
m.add_layer(ndvi, ndvi_vis, "NDVI (Landsat 8, 2015–2018)")
m.add_layer(zr.style(color='black', fillColor='00000000', width=1), {}, "Delhi Wards")
#m.add_colorbar(ndvi_vis, label='NDVI')
m

Map(center=[28.61, 77.23], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

### Code for Time lapse using Inspector tool

In [ ]:
roi = wards_fc.geometry()
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=landsat_ts,
    left_names=layer_names,
    right_names=layer_names,
    left_vis=landsat_vis,
    right_vis=landsat_vis,
)
Map.centerObject(roi, zoom=8)
Map